In [2]:
import os
import xarray as xr
import numpy as np
import netCDF4 as nc
import shutil
import subprocess
from math import radians, sin
from pyproj import Geod
from shapely.geometry import Point, LineString, Polygon
import glob
from joblib import Parallel, delayed
from tqdm import tqdm, trange
from myfunc import timer, run_command
import math

path = '/tera04/zhwei/xionghui/bedrock/'

Calculate the mask procedure include: 
1. mask area without adequate water (sum ET>P from 2003 to 2020)
2. mask area without woody vegetation (IGBP=1~9)
3. mask area without shallow bedrock (DTB<150cm)
the mask procedure4 need to calculate after the Dbedrock calculation.

In [ ]:
def mask_adequatewater() -> None:
    diff_file = os.path.join(path, 'diff', 'diff.nc')
    dir_path = os.path.join(path, 'mask_all', 'mask_adequatewater')
    os.makedirs(dir_path, exist_ok=True)
    output_sum = os.path.join(dir_path, 'diff_sum.nc4')
    output_interp = os.path.join(dir_path, 'diff_sum_interp.nc4')
    mask_file = os.path.join(dir_path, 'mask_adequatewater.nc4')
    run_command(f"cdo -f nc4 -P 48 timsum {diff_file} {output_sum}")
    run_command(f"gdalwarp -multi -wo NUM_THREADS=48 -ot Float32 -of netCDF -co FORMAT=NC4 -r bilinear -ts 86400 43200 -overwrite {output_sum} {output_interp}")
    run_command(f"cdo -f nc4 -z zip -setrtoc2,0,inf,nan,1 {output_interp} {mask_file}")
    print("mask_adequatewater completed")

def mask_woodyveg() -> None:
    dir_path = os.path.join(path, 'mask_all', 'mask_woodyveg')
    os.makedirs(dir_path, exist_ok=True)
    igbp_file = os.path.join(dir_path, 'global_igbp_15s_2020.nc')
    mask_file = os.path.join(dir_path, 'mask_woodyveg.nc4')
    run_command(f"cdo -f nc4 -z zip -setrtoc2,1,9,1,nan {igbp_file} {mask_file}")
    print("mask_woodyveg completed")

def mask_shallowbedrock() -> None:
    dir_path = os.path.join(path, 'mask_all', 'mask_shallowbedrock')
    os.makedirs(dir_path, exist_ok=True)
    dtb_file = os.path.join(dir_path, 'average_soil_and_sedimentary-deposit_thickness_remap_cm.nc')
    output_interp = os.path.join(dir_path, 'dtb_interp.nc4')
    mask_file = os.path.join(dir_path, 'mask_shallowbedrock.nc4')
    run_command(f"gdalwarp -multi -wo NUM_THREADS=48 -ot Float32 -of netCDF -co FORMAT=NC4 -r bilinear -ts 86400 43200 -overwrite {dtb_file} {output_interp}")
    run_command(f"cdo -f nc4 -z zip -setrtoc2,0,150,1,nan {output_interp} {mask_file}")
    print("mask_shallowbedrock completed")

In [ ]:
mask_adequatewater()
mask_woodyveg()
mask_shallowbedrock()

Calculate the Ssoil:
1. calculate the DTB stratification (0~5~15~30~60~100~150 cm) to align the SAWS stratification (0~5~15~30~60~100~200 cm)
2. use the vertical stratification (DTB and SAWS) to calculate the Ssoil

In [3]:
def Ssoil() -> None:
    layer = [0, 5, 15, 30, 60, 100, 150]
    dir_path = os.path.join(path, 'Ssoil')
    os.makedirs(dir_path, exist_ok=True)
    saws_path = os.path.join(dir_path, 'SAWS_Kosugi')
    ssoil_file = os.path.join(dir_path, f'Ssoil.nc4')

    def DTB_layer() -> None:
        dtb_file = os.path.join(dir_path, 'average_soil_and_sedimentary-deposit_thickness_remap_cm.nc')
        image = xr.open_dataset(dtb_file)
        s = image['Band1']
        for i in range(len(layer)-1):
            dtb_layer_file = os.path.join(dir_path, f'DTB_layer{i+1}.nc')
            delta_s = s - layer[i]
            delta_s = np.where(delta_s>(layer[i+1]-layer[i]), (layer[i+1]-layer[i]), delta_s)
            delta_s = np.where(delta_s<0, 0, delta_s)
            
            shutil.copyfile(f'{saws_path}/saws{i+1}.nc', dtb_layer_file)
            with nc.Dataset(dtb_layer_file, 'a') as file:
                s_var = file.variables['Band1']
                s_var[:,:] = delta_s      
        print("DTB_layer completed")

    DTB_layer()
    for i in range(len(layer)-1):
        dtb_layer_file = os.path.join(dir_path, f'DTB_layer{i+1}.nc')
        saws_layer_file = os.path.join(dir_path, f'saws{i+1}.nc')
        ssoil_layer_file = os.path.join(dir_path, f'Ssoil_layer{i+1}.nc4')
        run_command(f'ln -sf {saws_path}/saws{i+1}.nc {saws_layer_file}')
        run_command(f'cdo -f nc4 -z zip -mulc,10 -mul {saws_layer_file} {dtb_layer_file} {ssoil_layer_file}')
    filelist = [f'{dir_path}/Ssoil_layer{i+1}.nc4' for i in range(len(layer)-1)]
    filelistname = ' '.join(filelist)
    run_command(f'cdo -f nc4 -z zip -enssum {filelistname} {ssoil_file}')
    print("Ssoil completed")

In [4]:
Ssoil()

DTB_layer completed
Ssoil completed


In [5]:
layer = [0, 5, 15, 30, 60, 100, 150]
dir_path = os.path.join(path, 'Ssoil')
for i in range(len(layer)-1):
    print(i)
    ssoil_layer_file = os.path.join(dir_path, f'Ssoil_layer{i+1}.nc4')
filelist = [f'{dir_path}/Ssoil_layer{i+1}.nc4' for i in range(len(layer)-1)]
filelistname = ' '.join(filelist)
ssoil_file = os.path.join(dir_path, f'Ssoil.nc4')

print(f'cdo -f nc4 -z zip -enssum {filelistname} {ssoil_file}')


0
1
2
3
4
5
cdo -f nc4 -z zip -enssum /tera04/zhwei/xionghui/bedrock/Ssoil/Ssoil_layer1.nc4 /tera04/zhwei/xionghui/bedrock/Ssoil/Ssoil_layer2.nc4 /tera04/zhwei/xionghui/bedrock/Ssoil/Ssoil_layer3.nc4 /tera04/zhwei/xionghui/bedrock/Ssoil/Ssoil_layer4.nc4 /tera04/zhwei/xionghui/bedrock/Ssoil/Ssoil_layer5.nc4 /tera04/zhwei/xionghui/bedrock/Ssoil/Ssoil_layer6.nc4 /tera04/zhwei/xionghui/bedrock/Ssoil/Ssoil.nc4


In [ ]:
dir_path = os.path.join(path, 'mask_all', 'mask_adequatewater')
file1 = os.path.join(dir_path, 'mask_adequatewater.nc4')
file2 = os.path.join(dir_path, 'mask_adequatewater_0p1.nc4')
run_command(f"gdalwarp -multi -wo NUM_THREADS=48 -ot Float32 -of netCDF -co FORMAT=NC4 -r bilinear -ts 3600 1800 -overwrite {file1} {file2}")

In [ ]:
dir_path = os.path.join(path, 'mask_all', 'mask_woodyveg')
file1 = os.path.join(dir_path, 'mask_woodyveg.nc4')
file2 = os.path.join(dir_path, 'mask_woodyveg_0p1.nc4')
run_command(f"gdalwarp -multi -wo NUM_THREADS=48 -ot Float32 -of netCDF -co FORMAT=NC4 -r bilinear -ts 3600 1800 -overwrite {file1} {file2}")

In [ ]:
dir_path = os.path.join(path, 'mask_all', 'mask_shallowbedrock')
file1 = os.path.join(dir_path, 'mask_shallowbedrock.nc4')
file2 = os.path.join(dir_path, 'mask_shallowbedrock_0p1.nc4')
run_command(f"gdalwarp -multi -wo NUM_THREADS=48 -ot Float32 -of netCDF -co FORMAT=NC4 -r bilinear -ts 3600 1800 -overwrite {file1} {file2}")

In [ ]:
dir_path = os.path.join(path, 'Ssoil')
file1 = os.path.join(dir_path, 'Ssoil.nc4')
file2 = os.path.join(dir_path, 'Ssoil_0p1.nc4')
run_command(f"gdalwarp -multi -wo NUM_THREADS=48 -ot Float32 -of netCDF -co FORMAT=NC4 -r bilinear -ts 3600 1800 -overwrite {file1} {file2}")

In [ ]:
dir_path = os.path.join(path, 'mask_all', 'mask_adequatewater')
file1 = os.path.join(dir_path, 'mask_adequatewater.nc4')
file2 = os.path.join(dir_path, 'mask_adequatewater_0p1.nc4')
run_command(f"gdalwarp -multi -wo NUM_THREADS=48 -ot Float32 -of netCDF -co FORMAT=NC4 -r bilinear -ts 3600 1800 -overwrite {file1} {file2}")